In [0]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.models import load_model

from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.preprocessing import image
import matplotlib.image as mpimg

# **Import architecture and freeze layers**

In [9]:
#we import the architecture that we will use as the basis of our model (VGG16)

from keras.applications import VGG16
modelo_base = VGG16(weights = 'imagenet', include_top = False,  input_shape = (224, 224, 3))

for (i,k) in enumerate(modelo_base.layers):
  print(k.__class__.__name__,k.trainable)

#we print the active layers that we have in said model

print("--------------------------------------")

#we freeze the layers since we are going to make some modifications to said model
for (i,k) in enumerate(modelo_base.layers):
    k.trainable = False
    print(k.__class__.__name__,k.trainable)


InputLayer False
Conv2D True
Conv2D True
MaxPooling2D True
Conv2D True
Conv2D True
MaxPooling2D True
Conv2D True
Conv2D True
Conv2D True
MaxPooling2D True
Conv2D True
Conv2D True
Conv2D True
MaxPooling2D True
Conv2D True
Conv2D True
Conv2D True
MaxPooling2D True
--------------------------------------
InputLayer False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False
Conv2D False
Conv2D False
Conv2D False
MaxPooling2D False


# **Classification task**

In [10]:
#here we will organize the set with the images that we have easily with the ImageDataGenerator

#BD_training and BD_validation will contain the path where the images are located
#if necessary it should be modified with the path where the images are located on your computer
BD_entrenamiento = 'data_set/Train'
BD_validacion = 'data_set/Val'
GI_entrenamiento = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
 
GI_validacion = ImageDataGenerator(rescale=1./255)
 
 
generador_entrenamiento = GI_entrenamiento.flow_from_directory(BD_entrenamiento, target_size=(224, 224),
                                                    batch_size=64, class_mode='categorical')
 
generador_validacion = GI_validacion.flow_from_directory(BD_validacion, target_size=(224, 224),
                                                              batch_size=64, class_mode='categorical', shuffle=False)

Found 124 images belonging to 7 classes.
Found 35 images belonging to 7 classes.


# **Class Labels**

In [11]:
class_labels = generador_validacion.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)
print(len(class_labels))

{0: 'ben_afflek', 1: 'carlos', 2: 'elton_john', 3: 'jerry_seinfeld', 4: 'madonna', 5: 'meghan', 6: 'mindy_kaling'}
7


# **Auxiliary tool**

In [0]:

#auxiliary function to attach new layers in the top

def nuevas_layers(bottom_model, num_class, D=256):
    
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_class, activation = "softmax")(top_model)
    return top_model

# **Union layers**

In [13]:
#union of the input layer and the output layer
from keras.layers.normalization import BatchNormalization
from keras.models import Model

numero_classes = len(class_labels) 

Lineas_de_salida = nuevas_layers(modelo_base, numero_classes)

modelo_final = Model(inputs=modelo_base.input, outputs=Lineas_de_salida)

modelo_final.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

# **Setup and run model**

In [14]:
#checkpoint: is the parameter that will define where the model will be saved and under what conditions "val_loss, val_accuracy"
checkpoint = ModelCheckpoint(
    "code/modelos/model_1.h5", 
    monitor="val_loss", 
    mode="min", 
    save_best_only = True,
    verbose=1)

#early_stop: parameter that allows to control an early stop since the model has not improved its monitored variable "val_loss"
parada_temprana = EarlyStopping(monitor = 'val_loss',  min_delta = 0,  patience = 3, verbose = 1, restore_best_weights = True)

callbacks = [checkpoint]

#compile: is the way to configure the learning process before running the training,

modelo_final.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001), metrics = ['accuracy'])

Numero_Im_entrenamiento = 124
Numero_Im_validacion = 35
linas_de_process = 20
tamano_lote = 5
#tamano_lote or bachzise
Run_model = modelo_final.fit_generator(generador_entrenamiento, steps_per_epoch = Numero_Im_entrenamiento // tamano_lote, epochs = linas_de_process, 
                              callbacks = callbacks, validation_data = generador_validacion,
                              validation_steps = Numero_Im_validacion // tamano_lote)

Epoch 1/20
24/24 [==============================] - 29s 1s/step - loss: 6.8175 - accuracy: 0.3495 - val_loss: 1.3328 - val_accuracy: 0.5143

Epoch 00001: val_loss improved from inf to 1.33283, saving model to /content/drive/My Drive/Final_Project/Modelos/modelo_cara_slid.h5
Epoch 2/20
24/24 [==============================] - 22s 925ms/step - loss: 1.1574 - accuracy: 0.6351 - val_loss: 1.2749 - val_accuracy: 0.5429

Epoch 00002: val_loss improved from 1.33283 to 1.27488, saving model to /content/drive/My Drive/Final_Project/Modelos/modelo_cara_slid.h5
Epoch 3/20
24/24 [==============================] - 23s 942ms/step - loss: 0.7826 - accuracy: 0.7231 - val_loss: 0.7903 - val_accuracy: 0.6857

Epoch 00003: val_loss improved from 1.27488 to 0.79035, saving model to /content/drive/My Drive/Final_Project/Modelos/modelo_cara_slid.h5
Epoch 4/20
24/24 [==============================] - 23s 938ms/step - loss: 0.6477 - accuracy: 0.7809 - val_loss: 0.5485 - val_accuracy: 0.7714

Epoch 00004: val_